# Gli errori e le eccezioni

## Aspettarsi l’inaspettato: gestione delle Eccezioni (exceptions)

# Introduzione

Funzioni nella programmazione non Object-Oriented: 
- Controllavano rigorosamente gli input
- Restituivano valori inaspettati per indicare input o situazioni errate


Programmazione Object-Oriented:
- Gestione eccezioni: oggetti speciali per gli “errori” (tutte le eccezioni ereditano dall’oggetto BaseException)


Vedremo:
- Come causare un’eccezione 
- Come gestire un’eccezione quando capita
- Come gestire tipi diversi di eccezione in modi diversi
- Come ripulire la situazione quando capita un’eccezione 
- Creare nuovi tipi di eccezione
- Usare la sintassi delle eccezioni per gestire il flusso di controllo

### Riferimenti

Philips, Giridhar, Kasampalis - Python - Master the art of design patters - Packt ed.



# Rilanciare eccezioni

Le eccezioni diventano oggetti speciali quando sono gestite dentro il flusso di controllo di un programma.


In [1]:
print "ciao mondo"

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-1-66a718044c8f>, line 1)

In [ ]:
x = 5 / 0

In [ ]:
lst = [1,2,3]
print(lst[3])

In [ ]:
lst + 2

In [ ]:
lst.add

In [ ]:
d = {'a': 'hello'}
d['b']

In [ ]:
print(questa_non_e_una_variabile)


## Errori e eccezioni

Avrete notato che le precedenti eccezioni finiscono con Error.

Error e Exception sono quasi intercambiabili.

Error è semanticamente un po’ più grave ma il loro comportamente è identico

Exception (che estende BaseException) è la loro classe padre (superclass)

## Rilanciare un'eccezione

Prima di imparare come gestirle, vediamo come rilanciarle, ovvero come informare l'utente o la funzione chiamante che qualcosa non va. 

Non sarebbe comodo che i nostri programmi si comportino come Python?

Immaginiamo una nuova classe lista "SoloNumeriPariInteri"  a cui si possano aggiungere solo numeri interi pari...

In [ ]:
class SoloNumeriPariInteri(list):
    def append(self, integer):
        if not isinstance(integer, int):
            raise TypeError("Possono essere sommati solo numeri interi")
        if integer % 2:
            raise ValueError("Possono essere sommati solo numeri pari")
        super().append(integer)
e = SoloNumeriPariInteri()

La classe estende list (oggetto predefinito in Python) e ridefinisce (override) append.

Il nuovo metodo append verifica due condizioni:
- che l'input sia un integer, altrimenti rilancia TypeError
- che l'input sia pari, altrimenti rilancia ValueError

La parola chiave "raise" è semplicemente seguita dall'oggetto che viene rilanciato come exception. 

I due oggetti sono "costruiti" dalle classi built-in TypeError e ValueError ma potevano essere una eccezione creata "ad-hoc" (dopo vedremo come farlo)

In [ ]:
e.append(3)

In [ ]:
e.append(3.4)


<div class="alert alert-warning">
<b>Nota</b>
<br>
L'esempio è utile ai fini didattici ma la classe non è pensata bene.<br>
Infatti, ad esempio, risulta possibile assegnare un valore non valido usando gli indici.<br>
Per completare il lavoro occorrerebbe fare overload di altri metodi "interni", quelli con __ per intenderci<br>
<br>
</div>



In [ ]:
e.append(2)
e[0] = 1
print(e)

## Gli effetti di un'eccezione

Sembra stoppare l’esecuzione di un programma immediatamente. 

Le linee successive non sono eseguite


In [ ]:
def nessun_ritorno():
    print("Sto per rilanciare un’eccezione")
    raise Exception("Questo è sempre rilanciato")
    print("Questa linea non sarà mai eseguita")
    return "Non sarò restituito"

nessun_ritorno()

Inoltre, se una funzione chiama altra funzione che rilancia eccezione,
le linee successive alla chiamata NON vengono chiamate.

L'eccezione stoppa tutte le esecuzioni, a cascata, risalendo, fino a che è gestita o forzando l'uscita dell'interprete.

In [ ]:
def chiama_eccessore():
    print("chiama_eccessore inizia qui...")
    nessun_ritorno()
    print("eccezione rilanciata...")
    print("...quindi queste linee non sono eseguite")
    
chiama_eccessore()

Vedremo presto come poter gestire questi casi e reagire alle eccezioni, a qualsiasi livello.

Dallo stacktrace (output dell'exception) dal basso verso l'alto si vede la catena: dove l'eccezione è partita,
fino ad arrivare all'interprete che non sapendo cosa fare, rinuncia e mostra l'errore a video.


# Gestire le eccezioni

Si gestiscono racchiudendo la porzione di codice che potrebbe rilanciarle tra try e catch

In [ ]:
try:
    nessun_ritorno()
except:
    print("Ho catturato un'eccezione")
print("Eseguito dopo l'eccezione")

Sapevamo che nessun_ritorno() rilanciava un'eccezione e noi l'abbiamo catturata, continuando poi con l'esecuzione...

Notare l'indentazione del codice tra try e except

Il difetto del codice precedente è che cattura qualsiasi eccezione...

Cosa fare se volessimo trattare diversamente ad esempio TypeError rispetto a ZeroDivisionError?

In [ ]:
def cento_diviso(divisore):
    try:
        return 100 / divisore
    except ZeroDivisionError:
        return "Dividere per zero non è una buona idea!"

print(cento_diviso(0))
print(cento_diviso(50.0))
print(cento_diviso("hello"))

Si possono gestire più eccezioni con stesso codice, o con codice diverso

In [ ]:
def cento_diviso_v2(divisore):
    try:
        if divisore == 17:
            raise ValueError("17 è un numero sfortunato")
        return 100 / divisore
    except (ZeroDivisionError, TypeError):
        return "Inserire un numero, diverso da zero"

for val in (0, "ciao", 50.0, 17):

    print("Testiamo con {}:".format(val),end=" ")
    print(cento_diviso_v2(val))

L'eccezione derivata dal numero 17 non è lista tra le gestite, quindi non viene gestita.

Quindi se vogliamo gestire e fare cose diverse? (vedi distinzione tra ZeroDivisionError e TypeError sotto)

O se vogliamo fare in modo di fare qualcosa e poi rilanciarla al chiamante? (vedi ValueError sotto)

In [ ]:
def cento_diviso_v3(divisore):
    try:
        if divisore == 17:
            raise ValueError("17 è un numero sfortunato")
        return 100 / divisore
    except ZeroDivisionError:
        return "Inserire un numero diverso da zero"
    except TypeError:
        return "Inserire un numero"
    except ValueError:
        print("No, no, per favore non 17!")
        raise
        
for val in (0, "ciao", 50.0, 17):

    print("Testiamo con {}:".format(val),end=" ")
    print(cento_diviso_v3(val))

Mettendo in fila le eccezioni come nell'esempio precedente, solo la prima "associazione" viene eseguita, anche nel caso ce ne fossero di più. 

Ma qual è il caso di più associazioni?
Le eccezioni sono oggetti con una loro gerarchia, la maggior parte deriva da Exception (che a sua volta deriva da BaseException). 
Se nel codice gestiamo Exception prima di TypeError, verrà gestita solo Exception perchè TypeError è anche Exception per ereditarietà.

Questi ragionamenti sono utili quando vogliamo gestire delle eccezioni in modo particolare ma poi proteggerci in modo più generale da quelle rimanenti / generiche.



   


## Riferimento ad un eccezione

Qualche volta vogliamo avere il riferimento dell'oggetto eccezione, magari per poter accedere a degli attributi.
Per far questo si usa "as e" come si vede dopo.

In [ ]:
try:
    raise ValueError("Questo è un argomento")
except ValueError as e:
    print("Gli argomenti dell'eccezione sono ", e.args)


## finally and else

Ora vediamo come eseguire del codice indipendentemente dall'evento eccezione con le parole chiave finally e else

In [ ]:
import random

lista_con_alcune_eccezioni = [ZeroDivisionError, ValueError, TypeError, IndexError, None]

try:
    scelta = random.choice(lista_con_alcune_eccezioni)
    print("Rilancio... {}".format(scelta))
    if scelta:
        raise scelta("Un errore")
except ZeroDivisionError:
            print("Gestita ZeroDivisionError")
except ValueError:
            print("Gestita ValueError")
except TypeError:
            print("Gestita TypeError")
except Exception as e:
            print("Gestita eccezione di cui posso sapere il nome: %s" %(e.__class__.__name__))          
else:
    print("Questo codice è eseguito in assenza di eccezioni")
finally:
    print("Questo codice è eseguito sempre, per pulizia finale")

La parte di finally è eseguita in qualsiasi caso ed è da utilizzare per i task da effettuare alla fine dell'esecuzione, qualsiasi cosa accada (es. chisura file, chiusura connessione DB o di rete).

E' importante anche nelle funzioni che hanno un return nella parte try. Il codice nella parte finally verrà comunque eseguito prima del ritorno.

Se non ci sono eccezioni sono eseguite sia else che finally

except, else, e finally possono essere omessi (anche se else da solo non va bene).
Se si mettono: except prima, poi else e finally alla fine.clause at the end. 
L'ordine degli except va dall'eccezione più specifica alla più generica.

# La gerarchia delle eccezioni

La maggior parte delle eccezioni derivano da Exception (che però non è la base); eredita infatti da BaseException, come le due speciali SystemExit e KeyboardInterrupt.

SystemExit è rilanciata naturalmente alla fine del programma o dalla funzione sys.exit

KeyboardInterrupt è rilanciata a terminale dalla sequenza Ctrl+C

![Gerarchia delle eccezioni](./resources/eccezioni.png)

Quando si usa except: senza specificare il tipo di exception, si catturano tutte le sottoclassi di BaseException (comprese le due speciali).

Poichè normalmente vogliamo escluderle nel loro trattamento speciale meglio usare except con argomenti.
Se vogliamo catturare tutte le exceptions tranne SystemExit e KeyboardInterrupt, esplictamente scriviamo catch Exception.

Se invece vogliamo catturare anche le speciali, per chiarezza sempre consigliato except BaseException: 
così da far capire esplicitamente l'intento

# Definire le proprie eccezioni

E' molto semplice, basta estendere Exception e non è neppure necessario specificare altre informazioni

In [ ]:
class PrelievoNonValido(Exception):
    pass

raise PrelievoNonValido("Non hai abbastanza soldi sul conto")

Ovviemente la classe eccezione può essere più complessa e articolata.

Il metodo di inizializzazione Exception.__init__ method è progettato per accettare qualsiasi argomento e memorizzarlo in una tupla di nome args. 
Questo rende semplice la definizione senza la necessità di sovrescrivere (override) __init__

In [ ]:
class PrelievoNonValido(Exception):
    def __init__(self, saldo, importo):
        super().__init__("Il conto non ha EUR "+ str(importo))
        self.importo = importo
        self.saldo = saldo
    def sconfino(self):
        return self.importo - self.saldo

raise PrelievoNonValido(25, 60)        

In [ ]:
try:
    raise PrelievoNonValido(25, 60)
except PrelievoNonValido as e:
    print("Banca Python: ci dispiace ma la sua richiesa è superiore "
            "al saldo di "
            "EUR {}".format(e.sconfino()))

Avete visto come è stata gestita l'eccezione PrelievoNonValido.

E anche l'utilizzo di as seguita dal nome istanza dell'eccezion.
Per convenzione si usa e ma si potrebbe chiamare ex o pippo.

Ci sono molti motivi per definire "proprie" eccezioni:
- aggiungere info utili all'eccezione
- loggare
- in caso di framework / librerie / API per rilanciare eccezioni sensate ai caller, affinchè sia chiaro come sistemare l'errore o gestirlo

Le eccezioni non servono per gestire eventi eccezionali.
Molto spesso si tratta di scegliere il design...


In [ ]:
def dividi_con_exception(dividendo, divisore):
    try:
        print("{} / {} = {}".format(
            dividendo, divisore, dividendo / divisore * 1.0))
    except ZeroDivisionError:
        print("Non puoi dividere per zero!")

def dividi_con_if(dividendo, divisore):
    if divisore == 0:
        print("Non puoi dividere per zero!")
    else:
        print("{} / {} = {}".format(
            dividendo, divisore, dividendo / divisore * 1.0))

In [ ]:
dividi_con_exception(6,2)
dividi_con_exception(3,0)

In [ ]:
dividi_con_if(6,2)
dividi_con_if(3,0)

Anche negli casi di eccezione si potrebbe "proteggere" il codice con tante if, ma a parte la difficoltà di prevedere tutti i casi, il suggerimento è quello di eseguire il codice e gestire la situazione se qualcosa va male, anche per risparmiare codice e cicli CPU inseriti solo per gestire scenari rari

# Un ultimo esempio

Immaginiamo di gestire un inventario per un'azienda che vende gadget.
Quando un cliente fa un ordine di un prodotto, questo può essere disponibile (e quindi rimosso dall'inventario) oppure non disponibile (e questo evento è tutt'altro che eccezionale).

Cosa dovrebbe restituire la funzione "ordine"? -1, una stringa descrittiva, decisamente meglio un'eccezione tipo ProdottoEsauritoException

Definiamo con solo "docstring" l'oggetto e vediamo poi come usarlo...

In [ ]:
class Inventario:
    def blocca(self, prodotto):
        '''Seleziona e blocca il prodotto che sta per essere modificato.
        Questo metodo lo blocca per prevenire che sia aquistato contemporaneamente
        da più clienti.'''
        pass

    def sblocca(self, prodotto):
        '''Sblocca il prodotto così da permettere ad altri
        clienti di accedervi.'''
        pass

    def acquista(self, prodotto):
        '''Se il prodotto non esiste, eccezione ProdottoNonValidoException.
        Se il prodotto non è disponibile ProdottoEsauritoException.
        Altrimenti ritorna il numero di oggetti "prodotto" disponibili'''
        return 10


In [ ]:
tipo_prodotto = 'portachiave'
inv = Inventario()
inv.blocca(tipo_prodotto)
try:
    disponibili = inv.acquista(tipo_prodotto)
except ProdottoNonValidoException:
    print("Ci dispiace ma non trattiamo {}", format(tipo_prodotto))
except ProdottoEsauritoException:
    print("Ci dispiace ma {} è esaurito", format(tipo_prodotto))
else:
    print("Acquisto completato. Nell'inventario ancora "
            "{} oggetti di tipo {} sono disponibili".format(disponibili, tipo_prodotto))
finally:
    inv.sblocca(tipo_prodotto)